In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

import nltk

import Levenshtein

In [2]:
df_ipccreport_body = pd.read_csv("df_ipccreport_body.csv")
df_ipccreport_body_trunc = df_ipccreport_body[['text','geography_iso','author','document_name','date','types0','text_block_id']]
len(df_ipccreport_body)

16666

In [3]:
ipcc_report_names = pd.DataFrame(df_ipccreport_body.groupby('date_document_name')['text'].count())
#ipcc_reports.sort_values(['date_document_name'],ascending=False).head(60)
df_ipccreport_body_sfp = df_ipccreport_body[df_ipccreport_body['document_name'].str.contains("Summary for Policymakers")]
len(df_ipccreport_body_sfp)

498

In [4]:
df_ipccmentions_body = pd.read_csv("df_ipccmentions_body.csv")
df_ipccmentions_body_trunc = df_ipccmentions_body[['text','geography_iso','author','document_name','date','types0','text_block_id']]
len(df_ipccmentions_body)

18848

## Paragraphs to sentences

In [5]:
# find some way of uniquely identifying sentences that is not using the text

In [6]:
def para_to_sent(text_list):
    sent_list = []
    index_list = []
    for i in range(len(text_list)):
            text = text_list[i]
            sent_text = nltk.sent_tokenize(text)
            # Add sentences list to main list, and their index
            sent_list += sent_text
            for j in range(len(sent_text)):
                index_list += [i]
    print(len(text_list))
    print(len(sent_list))
    sent_df = pd.DataFrame(list(zip(sent_list, index_list)),
              columns=['Sentence','Row_index'])
    return sent_df

In [7]:
# df_ipccreport_body
df_ipccreport_body_sent_df = para_to_sent(df_ipccreport_body['text'])
df_ipccreport_body_sent_df = df_ipccreport_body_sent_df[(df_ipccreport_body_sent_df['Sentence'].str.count(' ') + 1)>=5]
df_ipccreport_body_sent_df = df_ipccreport_body_sent_df.reset_index(drop=True)
len(df_ipccreport_body_sent_df)

16666
66991


56077

In [8]:
# df_ipccmentions_body
df_ipccmentions_body_sent_df = para_to_sent(df_ipccmentions_body['text'])
df_ipccmentions_body_sent_df = df_ipccmentions_body_sent_df[(df_ipccmentions_body_sent_df['Sentence'].str.count(' ') + 1)>=5]
df_ipccmentions_body_sent_df = df_ipccmentions_body_sent_df.reset_index(drop=True)
len(df_ipccmentions_body_sent_df)

18848
52656


49541

## Levenshtein distance

In [9]:
def compute_levenshtein(a,b):
    dist = Levenshtein.distance(a, b)
    rat = Levenshtein.ratio(a, b)
    mb = Levenshtein.matching_blocks(Levenshtein.editops(a,b), a, b)
    matching = []
    for x in mb:
        # Only include matching strings of a certain length
        if x[2]>5:
           match = a[x[0]:x[0]+x[2]]
           matching.append(match)
    #print(rat)
    return dist, rat, matching

In [31]:
a_list = []
a_idx_list = []
b_list = []
b_idx_list = []
dist_list = []
ratio_list = []
matching_list = []

#for i in tqdm(range(10000)):
#    for j in range(10000):

for i in tqdm(range(len(df_ipccreport_body_sent_df))):
    for j in range(len(df_ipccmentions_body_sent_df)):
        #print(i)
        #print(j)

        report_sent = df_ipccreport_body_sent_df['Sentence'][i]
        report_sent_ri = df_ipccreport_body_sent_df['Row_index'][i]

        mentions_sent = df_ipccmentions_body_sent_df['Sentence'][j]
        mentions_sent_ri = df_ipccmentions_body_sent_df['Row_index'][j]

        dist, ratio, matching = compute_levenshtein(report_sent,mentions_sent)
        
        if ratio > 0.6:
            a_list.append(report_sent)
            a_idx_list.append(report_sent_ri)
            b_list.append(mentions_sent)
            b_idx_list.append(mentions_sent_ri)
            
            dist_list.append(dist)
            ratio_list.append(ratio)
            matching_list.append(matching)
            # print(report_sent)
            # print(mentions_sent)
            # print(dist)
            # print(ratio)
            # print(matching)
            # print('\n')  

100%|██████████████████████████████████| 56077/56077 [44:07:52<00:00,  2.83s/it]


In [ ]:
# Also need the dataframe from IPCC report to itself

In [32]:
# Combine into dataframe
sent_sim = pd.DataFrame(list(zip(a_list, a_idx_list, b_list, b_idx_list, dist_list, ratio_list, matching_list)),
              columns=['Text (IPCC report)','Text (IPCC report) - Row Index','Text (IPCC mentions)','Text (IPCC mentions) - Row Index', 'Distance', 'Ratio', 'Matching Text'])

In [34]:
len(sent_sim)
#pd.set_option('display.max_colwidth', 500)

5785

In [35]:
sent_sim.to_csv("sent_sim20240423.csv")

In [ ]:
# Change all to lower then perform the matching?